sos run nbs/seqlink_sos.ipynb --cwd data/wg20220316 --fam_path data/new_trim_ped_famless17_no:xx.fam --vcf_path /mnt/mfs/statgen/alzheimers-family/linkage_files/geno/full_sample/vcf/full_sample.vcf.gz --anno_path MWE/annotation --pop_path data/full_sample_fam_pop.txt

In [ ]:
[seqlink (phasing haps)]
# Work directory where output will be saved to
parameter: cwd = path
# Fam file
parameter: fam_path = path
# VCF file
parameter: vcf_path = path
# Phenotype path
parameter: anno_path = path
# Sample file path, for bgen format
parameter: pop_path = path
parameter: walltime = '24h'
parameter: mem = '100G'
# Load all regions of interest. Each item in the list will be a region: (chr, start, end)
chrom = list(range(11,19))+[22]
input: fam_path, vcf_path, anno_path, pop_path, for_each = 'chrom'
output: f'{cwd:a}/chr{_chrom}test'
task: walltime = walltime, mem = mem, tags = f'{step_name}_{_output:bn}'
bash: expand = '${ }', stderr = f'{_output:n}.stderr', stdout = f'{_output:n}.stdout'
    
    echo "start"
    seqlink --fam ${fam_path} --vcf ${vcf_path} \
    --anno '${anno_path:a}/EFIGA_NIALOAD_chr${_chrom}.hg38.hg38_multianno.csv' \
    --pop ${pop_path} \
    -o ${_output}  \
    -f 'MERLIN' --build 'hg38' --freq 'AF' --bin 1 --maf-cutoff 0.05 --jobs 1